In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [9]:
# SCRAPING FUNCTION
# This function parses rotogrinders website for Player Names, Teams, and Positions for MLB data
# Input is a url that has a GET result for the rotogrinders url
# example url = 'https://rotogrinders.com/game-stats/mlb-pitcher?site=draftkings&range=season'
def parse_nfl_stat_data(url):
    # Initialize driver object
    driver = webdriver.Firefox()   # Establish the driver, we are using FireFox in this case
    driver.get(url) 

    try:
        # Find the ALL button, to show all results 
        page_buttons = driver.find_elements_by_class_name("page")
    
        all_button = page_buttons[len(page_buttons)-1]
        # Click the button
        driver.execute_script("arguments[0].click();", all_button)
    except:
        pass
    
    # Find the table we want
    table = driver.find_element_by_id('game-stats-table') 
    # Get the stats table
    table = driver.find_element_by_id('game-stats-table') 
    
    df = pd.DataFrame()
    
    # Loop through each column
    for col in table.find_elements_by_class_name('rgt-col'):
        # Get the column header for each column and append
        for header in col.find_elements_by_class_name('rgt-hdr'):
            # Grabbing all players names into a list 
            div_tags = col.find_elements_by_tag_name('div')
            column_data = [tag.text for tag in div_tags]
            
            df[column_data[0]] = column_data[1:]
        
    return(df)

# Turns out that the RotoGrinders database is very limited, especially for NFL preseason, so using a more robust source, The Football Database

In [25]:
url = 'https://www.footballdb.com/players/current.html?letter=A'

In [63]:
test = parse_nfl_stat_data(url)

In [41]:
team_names_dict = {'Arizona':'ARI',
                    'Atlanta':'ATL',
                    'Baltimore':'BAL',
                    'Buffalo':'BUF',
                    'Carolina':'CAR',
                    'Chicago':'CHI',
                    'Cincinnati':'CIN',
                    'Cleveland':'CLE',
                    'Dallas':'DAL',
                    'Denver':'DEN',
                    'Detroit':'DET',
                    'Green Bay':'GB',
                    'Houston':'HOU',
                    'Indianapolis':'IND',
                    'Jacksonville':'JAX',
                    'Kansas Cit':'KC',
                    'Las Vegas':'LV',
                    'LA Chargers':'LAC',
                    'LA Rams':'LAR',
                    'Miami':'MIA',
                    'Minnesota':'MIN',
                    'New England':'NE',
                    'New Orleans':'NO',
                    'NY Giants':'NYG',
                    'NY Jets':'NYJ',
                    'Philadelphia':'PHI',
                    'Pittsburgh':'PIT',
                    'San Francisco':'SF',
                    'Seattle':'SEA',
                    'Tampa Bay':'TB',
                    'Tennessee':'TEN',
                    'Washington':'WAS'}

In [56]:
test = test[0]

test

'Abdullah, Ameer RB Minnesota Nebraska'

In [65]:
#[string.replace(team_names_dict) for string in test]

''.join([team_names_dict.get(i, i) for i in test])


#for pat, repl in dic.items():
#    df.Text = df.Text.str.replace(pat, repl)
    
test = pd.Series(test).replace(team_names_dict, regex=True)

In [68]:
values = [row.replace(' ','|') for row in test]



In [69]:
values

['Abdullah,|Ameer|RB|MIN|Nebraska',
 'Aboushi,|Oday|OG|LAC|Virginia',
 'Abram,|Johnathan|DB|LV|Mississippi|State',
 'Abrams,|Delrick|DB|ATL|Colorado',
 'Acy,|DeMarkus|DB|PIT|Missouri',
 'Adams,|Andrew|DB|PHI|Connecticut',
 'Adams,|Davante|WR|GB|Fresno|State',
 'Adams,|Jamal|DB|SEA|Louisiana|State',
 'Adams,|Jerell|TE|TB|South|CAR',
 'Adams,|Josh|RB|NYJ|Notre|Dame',
 'Adams,|Matthew|LB|IND|HOU',
 'Adams,|Montravius|DT|NE|Auburn',
 'Adams,|Myles|DT|SEA|Rice',
 'Adams,|Paul|OT|TEN|Missouri',
 'Adams,|Rodney|WR|CHI|South|Florida',
 'Adams,|Tyrell|LB|BUF|West|Georgia',
 'Adderley,|Nasir|DB|LAC|Delaware',
 'Addison,|Mario|DE|BUF|Troy|State',
 'Adebo,|Paulson|DB|NO|Stanford',
 'Adeniji,|Hakeem|OG|CIN|Kansas',
 'Adeniyi,|Olasunkanmi|LB|TEN|Toledo',
 'Adeoye,|Aaron|LB|BAL|Southeast|Missouri|State',
 'Adomitis,|Graham|TE|IND|Princeton',
 'Agholor,|Nelson|WR|NE|Southern|California',
 'Agim,|McTelvin|DT|DEN|Arkansas',
 'Agnew,|Jamal|WR|JAX|San|Diego',
 'Ahmed,|Salvon|RB|MIA|WAS',
 'Aiyuk,|Brandon|

In [37]:
# SCRAPING FUNCTION
# This function parses rotogrinders website for Player Names, Teams, and Positions for MLB data
# Input is a url that has a GET result for the rotogrinders url
# example url = 'https://rotogrinders.com/game-stats/mlb-pitcher?site=draftkings&range=season'
def parse_nfl_stat_data(url):
    # Initialize driver object
    driver = webdriver.Firefox()   # Establish the driver, we are using FireFox in this case
    driver.get(url) 
    
    #df = pd.DataFrame()
    
    # Get all the table data
    #data = driver.find_elements_by_class_name('td')
    data = driver.find_elements_by_class_name('tr')
    # Parse out the values from the table data tags
    values = [tags.text for tags in data]
    
    # Replace the team names to something more standardized, abbreviations
    values = pd.Series(values).replace(team_names_dict, regex=True)
    
    # Replace all whitespace NOT after a comma with a | sign
    values = values.replace(' ', '|')
        
    return(values)

In [14]:
qb_df = parse_nfl_stat_data('https://rotogrinders.com/game-stats/nfl-qb?site=draftkings&range=last-two')
qb_df = qb_df[['Player','Team','Pos']]

In [17]:
print(len(qb_df))
qb_df.head()

74


,Player,Team,Pos
0,Aaron Rodgers,GBP,QB
1,Tom Brady,TBB,QB
2,Joe Flacco,PHI,QB
3,Matthew Stafford,LAR,QB
4,Matt Ryan,ATL,QB


In [7]:
flex_df = parse_nfl_stat_data('https://rotogrinders.com/game-stats/nfl-flex?site=draftkings&range=last-two')
flex_df = flex_df[['Player','Team','Pos']]

In [19]:
print(len(flex_df))
flex_df.head()

514


,Player,Team,Pos
0,Mark Ingram,HOU,RB
1,Rob Gronkowski,TBB,TE
2,Marcedes Lewis,GBP,TE
3,Jimmy Graham,CHI,TE
4,Jared Cook,LAC,TE


In [11]:
kicker_df = parse_nfl_stat_data('https://rotogrinders.com/game-stats/nfl-kicker?site=draftkings&range=last-two')
kicker_df = kicker_df[['Player','Team','Pos']]

In [21]:
print(len(kicker_df))
kicker_df.head()

35


,Player,Team,Pos
0,Mason Crosby,GBP,K
1,Ryan Succop,TBB,K
2,Robbie Gould,SFO,K
3,Nick Folk,NEP,K
4,Matt Prater,ARI,K


In [13]:
# CONCAT THE DATAFRAMES TO CREATE THE MASTER

In [22]:
master_df = pd.concat([qb_df, flex_df, kicker_df])

In [23]:
print(len(master_df))
master_df.head()

623


,Player,Team,Pos
0,Aaron Rodgers,GBP,QB
1,Tom Brady,TBB,QB
2,Joe Flacco,PHI,QB
3,Matthew Stafford,LAR,QB
4,Matt Ryan,ATL,QB


In [24]:
data_dir = 'C:/Users/ahote/Documents/dk_slate_study_tool/dash/nfl/assets/'
master_df.to_csv(data_dir + 'nfl_player_team_pos_df.csv', index=False)